# Graph Scheduling Operations

Examples of scheduling directives used in graph (and other) processing

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run prelude.py

## Graph Inputs

In [ ]:
#
# Function to create graph inputs
#

def create_inputs(display=True):
    
    # Adjacency matrix - Ranks "S" (source) and "D" (destination)

    a = Tensor.fromUncompressed([ "S", "D"],
                                [ [ 0, 1, 1, 0, 0, 0, 0, 1 ],
                                  [ 0, 0, 1, 1, 0, 0, 1, 0 ],
                                  [ 0, 0, 0, 1, 1, 0, 0, 0 ],
                                  [ 0, 0, 0, 0, 1, 1, 0, 0 ],
                                  [ 1, 0, 0, 0, 0, 1, 0, 0 ],
                                  [ 0, 1, 0, 0, 0, 0, 1, 0 ],
                                  [ 0, 0, 0, 1, 0, 1, 0, 0 ],
                                  [ 1, 1, 0, 0, 0, 1, 0, 0 ] ])


    print("Adjacency Matrix")
    displayTensor(a)

    return (a)



# Uniform coordinate space tiling of two ranks (edge blocking)


In [ ]:
# Create inputs

a = create_inputs()

a_s = a.getRoot()

print("Graph")
displayGraph(a_s)

In [ ]:
# Split top rank(source rows) uniformly in coordinate space creating new rank (source tiles)

tile_size = 4

a_s = a_s.splitUniform(tile_size)

print(f"Source nodes split uniformly by {tile_size} coordinates")
displayTensor(a_s)

In [ ]:
# Split thrid rank (destination columns) in coordinate space creating new rank (destination tiles)

a_s.splitUniformBelow(4, depth=1)

print(f"Destination nodes split uniformly by {tile_size} coordinates")
displayTensor(a_s)

In [ ]:
# Swap second rank (source rows) with third rank (destination tiles)
# Result is:
#   - Source tiles
#   - Destination tiles
#   - Source rows
#   - Destination columns

a_s.swapRanksBelow()

print(f"Partitioned into {tile_size}x{tile_size} tiles")
displayTensor(a_s)

In [ ]:
# Display the graph for each tile - source nodes and destination nodes each in their own coordinate space tile

for st, a_dt in a_s:
    for dt, tile in a_dt:
        print(f"Graph tile ({st}, {dt})")
        displayGraph(tile)

# Top rank decreasing cluster size partitioning (ETWC)

In [ ]:
# Create inputs

a = create_inputs()

a_s = a.getRoot()



In [ ]:
# Group by 3's then the remainder by 1s

cluster_sizes = [3, 1]

partition_counts = []
remainder_sizes = []
total_sizes = []

remaining = len(a_s)

for size in cluster_sizes:
    # Number of clusters in this partition
    partition_counts.append(remaining // size)

    # Total number of coordinates for this cluster size
    total_sizes.append(partition_counts[-1] * size)

    # Number of coordinates left after this cluster size
    remainder_sizes.append(remaining % size)
    
    # Number of unallocated coordinates
    remaining = remainder_sizes[-1]

print(f"Cluster sizes = {cluster_sizes}")
print(f"Partition counts = {partition_counts}")
print(f"Total sizes = {total_sizes}")

splits = []

for size, count in zip(cluster_sizes, partition_counts):
    splits += [size]*count

print("\n")
print(f"Splits = {splits}")

In [ ]:
# Split according to splits

a_s_split = a_s.splitUnEqual(splits)

print(f"Graph split into groups of sizes of {splits}")
displayTensor(a_s_split)

In [ ]:
# Extra optional step in which cluster sizes are divided at a new rank

a_s_split2 = a_s_split.splitUnEqual(partition_counts)
displayTensor(a_s_split2)



In [ ]:
# Display the graph for each cluster

for st, a_dt in a_s_split2:
    for dt, tile in a_dt:
        print(f"Graph tile ({st}, {dt}) - has {len(tile)} source nodes")
        displayGraph(tile)

# Split first rank by position. Flatten and split by position again (WC)

In [ ]:
# Create inputs

a = create_inputs()

a_s = a.getRoot()


In [ ]:
s_partitions = 3
d_per_group = 4

a_s_split = a_s.splitEqual(s_partitions)

print(f"Graph split into {s_partitions} source nodes per partition")
displayTensor(a_s_split)

a_s_split.flattenRanksBelow()
a_s_split.splitEqualBelow(d_per_group)
a_s_split.unflattenRanksBelow(depth=1)

print("Graph split again with {d_per_group} destinations per group")
displayTensor(a_s_split)

In [ ]:
# Display the graph for each group - each group should have 4 edges

for st, a_dt in a_s_split:
    for dt, tile in a_dt:
        print(f"Graph tile ({st}, {dt})")
        displayGraph(tile)

# Degree based splitting

In [ ]:
# Create inputs

a = create_inputs()

a_s = a.getRoot()

In [ ]:
threshold = 2

a_s_big = Fiber()
a_s_small = Fiber()
ad = Fiber([0, 1], [a_s_big, a_s_small])

for s, (a_d) in a_s:
    if len(a_d) > threshold:
        print(f"Adding big {s}, {a_d}")
        a_s_big.append(s, a_d)
    else:
        print(f"Adding small {s}, {a_d}")
        a_s_small.append(s, a_d)

print("\n")
print("Graph split into nodes with > {threshold} destinations and <= {threshold} destinations")
displayTensor(ad)


Graph drawing

In [ ]:
# Display the graph for group - buckets should have >2 or <=2 edges

for bucket, tile in ad:
    print(f"Graph bucket: {bucket}")
    displayGraph(tile)

## Testing area

For running alternative algorithms